In [12]:
from scgenome.jointcnmodels import get_variances
from scgenome import simulation, utils
import pandas as pd
import numpy as np

N_SAMPLE = 4
N_BIN = 10
MAX_CN = 7

cn_mat = simulation.cn_mat_poisson(N_SAMPLE, N_BIN, max_cn=MAX_CN, seed=5)
cn_data = utils.cn_mat_to_cn_data(utils.cn_mat_as_df(cn_mat,chr_names=["1","2"]))
cn_data["state"] = cn_data["copy"] + np.abs(np.random.normal(size=cn_data.shape[0]))
display(cn_mat)
display(cn_data.head())

def nget_variances(cn_data, matrix_data, n_states=MAX_CN):
    cell_state_var = cn_data[['cell_id', 'state', 'copy']].dropna().groupby(
        ['cell_id', 'state'])['copy'].var().rename('copy_var').reset_index()
    variances = cell_state_var.set_index(['state', 'cell_id'])[
        'copy_var'].unstack()
    #variances = variances.reindex(columns=matrix_data['reads'].columns,
    variances = variances.reindex(columns=matrix_data['copy'].columns,
                                  index=range(n_states)).fillna(0.05).T
    variances = variances.values
    variances[variances < 0.001] = 0.001
    return variances

nget_variances(cn_data, cn_mat)

array([[0, 0, 1, 0, 2, 2, 2, 3, 3, 2],
       [1, 0, 0, 1, 1, 1, 1, 1, 1, 0],
       [2, 2, 2, 4, 3, 2, 1, 0, 0, 0],
       [2, 3, 3, 3, 3, 0, 4, 4, 4, 4]])

,chr,bin,cell_id,copy,start,end,state
0,1,0,cell0,0,0,9,0.003289
1,1,1,cell0,0,10,19,0.105930
2,1,2,cell0,1,20,29,1.793053
3,1,3,cell0,0,30,39,0.631572
4,1,4,cell0,2,40,49,2.006195


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [19]:
matrix_data, measurement, cell_ids = utils.cn_data_to_mat_data_ids(cn_data, value_ids=["state","copy"])
matrix_data["copy"]
#def cn_data_to_mat_data_ids(cn_data, data_id=CN_DATA_ID, cell_id=CELL_ID,
#                            index_ids=INDEX_IDS, value_ids=VALUE_IDS):

cell_id    cell0  cell1  cell2  cell3
chr start                            
1   0          0      1      2      2
    10         0      0      2      3
    20         1      0      2      3
    30         0      1      4      3
    40         2      1      3      3
2   50         2      1      2      0
    60         2      1      1      4
    70         3      1      0      4
    80         3      1      0      4
    90         2      0      0      4

In [5]:
from scgenome.simulation import many_poisson_bicluster, do_naive_hc, \
    pairwise_distance
from treeswift.Tree import read_tree_linkage

OUTPUT_FP = "/Users/massoudmaher/data/pois_ran_walk.json"
#OUTPUT_FP = "/work/shah/maherm/100t_pois_ran_walk.json"
TRIALS_PER_SET = 1
SAMPLES_PER_CLUSTER = [8]
NUM_BIN = [100, 500]
MAX_CN = [4]
#ALPHA = [0.01, 0.05, 0.3, 0.6, 0.9]
ALPHA = [0.01, 0.6, 0.9]
INIT_LAMBDAS = [(3, 1)]
JUMP_LAMBDAS = [(1, 0.1)]


def plinkage_to_tree(plinkage):
    cols = ["i", "j", "dist", "merge_count"]
    linkage = plinkage.loc[:, cols].to_numpy().astype("float")
    return read_tree_linkage(linkage)


print("Starting simulations")
sims = many_poisson_bicluster(TRIALS_PER_SET, SAMPLES_PER_CLUSTER, NUM_BIN,
                              MAX_CN, ALPHA, INIT_LAMBDAS, JUMP_LAMBDAS,
                              num_cores=8)
do_naive_hc(sims, "cityblock")

sims["naive_tree"] = sims["naive_linkage"].apply(read_tree_linkage)
sims["bhc_tree"] = sims["plinkage"].apply(plinkage_to_tree)
sims["naive_pwd"] = sims["naive_tree"].apply(pairwise_distance)
sims["bhc_pwd"] = sims["bhc_tree"].apply(pairwise_distance)

sims = sims.drop(columns=["naive_tree"])
sims = sims.drop(columns=["bhc_tree"])
sims.to_json(OUTPUT_FP)
sims.head()

Starting simulations
[                                        ] | 0% Completed |  0.0s(16, 100)
(16, 100)
(16, 100)
[                                        ] | 0% Completed |  0.1s(16, 500)
(16, 500)
[                                        ] | 0% Completed |  0.3s(16, 100)
(16, 100)
(16, 100)
[                                        ] | 0% Completed |  0.6s
(16, 500)
(16, 500)


ValueError: ('cannot copy sequence with size 7 to array axis with dimension 1', 'occurred at index 0')

Traceback
---------
  File "/Users/massoudmaher/Documents/Code/scgenome/venv/lib/python3.7/site-packages/dask/local.py", line 233, in execute_task
    result = _execute_task(task, data)
  File "/Users/massoudmaher/Documents/Code/scgenome/venv/lib/python3.7/site-packages/dask/core.py", line 119, in _execute_task
    return func(*args2)
  File "/Users/massoudmaher/Documents/Code/scgenome/venv/lib/python3.7/site-packages/dask/optimization.py", line 1059, in __call__
    return core.get(self.dsk, self.outkey, dict(zip(self.inkeys, args)))
  File "/Users/massoudmaher/Documents/Code/scgenome/venv/lib/python3.7/site-packages/dask/core.py", line 149, in get
    result = _execute_task(task, cache)
  File "/Users/massoudmaher/Documents/Code/scgenome/venv/lib/python3.7/site-packages/dask/core.py", line 119, in _execute_task
    return func(*args2)
  File "/Users/massoudmaher/Documents/Code/scge